In [1]:
from src import API,Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

INFO - Logger configured


In [11]:

#API call
callApi=API()
start="2023-06-22 11:34:00.000000"
end="2023-06-22 11:50:35.000000"
raw_data=callApi.getRawDataForCartoWear('C77C2F92664E',pd.to_datetime(start),pd.to_datetime(end))

#Preprocessing
sampling_time=2
filter_window='20S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_Filter(filter_window))\
        .process()
rssi_df

INFO - data Cleaning
INFO - cleaned, sorting
INFO - sorted, filtering
INFO - filtered


,timestamp,macModule,rssi
0,2023-06-22 09:34:01.500,A8032A311F6A,-70.0
1,2023-06-22 09:34:01.500,A8032A311FAA,-77.0
2,2023-06-22 09:34:04.000,A8032A311F96,-74.0
3,2023-06-22 09:34:05.500,A8032A311F6A,-70.0
4,2023-06-22 09:34:05.500,A8032A311FAA,-77.0
...,...,...,...
924,2023-06-22 09:50:30.500,A8032A311DAA,-76.0
925,2023-06-22 09:50:31.500,A8032A311F6A,-73.0
926,2023-06-22 09:50:33.500,A8032A311FAA,-74.0
927,2023-06-22 09:50:34.000,A8032A311F96,-75.0


In [12]:
def get_strongest_rssi(df,timestamp,n):
    """
    Return the n strongest rssi values at a given timestamp
    """
    return df.loc[timestamp].sort_values(ascending=False).head(n)

In [13]:
rssi_viewer(rssi_df)